In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data_path = "/root/data/"

df_fake = pd.read_csv(data_path + "Fake.csv")
df_true = pd.read_csv(data_path + "True.csv")
df_fake['label'] = 1  # Fake news label
df_true['label'] = 0   # Real news label
df_fake.head()

,title,text,subject,date,label
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",1
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",1
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",1
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",1
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",1


In [2]:
df_true.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",0
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",0
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",0
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",0
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",0


In [3]:
df = pd.concat([df_true,df_fake])
df = df.drop(['title','subject','date'],axis=1)
df.head()
df.tail()

,text,label
23476,21st Century Wire says As 21WIRE reported earl...,1
23477,21st Century Wire says It s a familiar theme. ...,1
23478,Patrick Henningsen 21st Century WireRemember ...,1
23479,21st Century Wire says Al Jazeera America will...,1
23480,21st Century Wire says As 21WIRE predicted in ...,1


In [4]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import re


nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

def load_glove_vectors(glove_file_path):
    word_vectors = {}
    with open(glove_file_path, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = np.array(values[1:], dtype='float32')
            word_vectors[word] = vector
    return word_vectors

def preprocess_text(text):
    text = text.lower()                              # Convert text to lowercase
    text = re.sub(r'[^a-zA-Z\s]', '', text)          # Remove special characters and numbers
    tokens = word_tokenize(text)                     # Tokenization
    stop_words = set(stopwords.words('english'))     # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    
    lemmatizer = WordNetLemmatizer()                 # Lemmatization  ex: running => run (not nesscessary, but i want to test)
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens

glove_path = "/root/data/glove/"
word_vectors = load_glove_vectors(glove_path + "glove.6B.100d.txt")


df['clean_text'] = df['text'].apply(preprocess_text)
df.head()



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,text,label,clean_text
0,WASHINGTON (Reuters) - The head of a conservat...,0,"[washington, reuters, head, conservative, repu..."
1,WASHINGTON (Reuters) - Transgender people will...,0,"[washington, reuters, transgender, people, all..."
2,WASHINGTON (Reuters) - The special counsel inv...,0,"[washington, reuters, special, counsel, invest..."
3,WASHINGTON (Reuters) - Trump campaign adviser ...,0,"[washington, reuters, trump, campaign, adviser..."
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,0,"[seattlewashington, reuters, president, donald..."


In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

def tokens_to_vectors(tokens):
    vectors = []
    for token in tokens:
        if token in word_vectors:
            vectors.append(word_vectors[token])
    if not vectors: # return none if no valid word vectors found 
        return None
    return np.mean(vectors, axis=0)

df['doc_vector'] = df['clean_text'].apply(tokens_to_vectors)
df = df.dropna()

X_train, X_test, y_train, y_test = train_test_split(df['doc_vector'].tolist(), df['label'], test_size=0.2, random_state=42)
df.head()

,text,label,clean_text,doc_vector
0,WASHINGTON (Reuters) - The head of a conservat...,0,"[washington, reuters, head, conservative, repu...","[-0.07795639, 0.14429979, 0.26982972, -0.25392..."
1,WASHINGTON (Reuters) - Transgender people will...,0,"[washington, reuters, transgender, people, all...","[0.05190649, -0.07784701, 0.12894869, -0.06527..."
2,WASHINGTON (Reuters) - The special counsel inv...,0,"[washington, reuters, special, counsel, invest...","[-0.09237187, -0.063385196, 0.2864152, -0.2219..."
3,WASHINGTON (Reuters) - Trump campaign adviser ...,0,"[washington, reuters, trump, campaign, adviser...","[-0.096482284, -0.17997286, 0.2508118, -0.0995..."
4,SEATTLE/WASHINGTON (Reuters) - President Donal...,0,"[seattlewashington, reuters, president, donald...","[-0.026239133, 0.07124637, 0.22610407, -0.1405..."


In [6]:
print(df['doc_vector'][0].to_list())

[array([-7.79563934e-02,  1.44299790e-01,  2.69829720e-01, -2.53925264e-01,
        1.40237972e-01, -9.83903110e-02, -2.16529578e-01,  9.37424004e-02,
       -3.74789387e-02,  4.11572866e-02, -5.93027472e-02, -4.50058170e-02,
       -4.08024862e-02, -8.83621275e-02, -4.68307398e-02, -2.55840719e-01,
        1.29539207e-01,  4.03240621e-02, -3.54152948e-01,  3.57165150e-02,
        1.92550048e-01, -8.38312134e-03,  8.49788859e-02,  2.08849000e-04,
       -2.33675376e-01, -1.37680680e-01, -2.03800529e-01, -3.84860039e-01,
       -1.26006201e-01, -1.22342862e-01,  1.72383264e-01,  4.30866629e-01,
        1.60489492e-02,  6.08795462e-03, -1.06554195e-01,  2.33929768e-01,
       -2.78121307e-02,  3.03712785e-02, -5.72155938e-02,  8.88695568e-02,
       -3.97056818e-01, -3.68877620e-01,  2.45969057e-01, -1.37277534e-02,
       -1.31840691e-01, -2.45733425e-01,  1.72431488e-02, -3.05981070e-01,
       -1.79011732e-01, -5.79672217e-01,  6.76574260e-02, -1.86401501e-01,
        1.15239732e-01, 

In [7]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report



knn_classifier = KNeighborsClassifier(n_neighbors=10)  
knn_classifier.fit(X_train, y_train)

y_pred = knn_classifier.predict(X_test)


accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)
print('----------------')
print('Confusion matrix')
print(confusion_matrix(y_pred,y_test))
print('----------------')
print('Classification report')
print(classification_report(y_pred,y_test))


Accuracy: 0.9246265278406519
----------------
Confusion matrix
[[4150  519]
 [ 147 4020]]
----------------
Classification report
              precision    recall  f1-score   support

           0       0.97      0.89      0.93      4669
           1       0.89      0.96      0.92      4167

    accuracy                           0.92      8836
   macro avg       0.93      0.93      0.92      8836
weighted avg       0.93      0.92      0.92      8836



In [8]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense

height = 100
width = 1
channels = 1

X_train = np.array(X_train)
X_test = np.array(X_test)

X_train = np.squeeze(X_train)
X_test = np.squeeze(X_test)


print(X_train.shape)



model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(100, 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))  


model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()


history = model.fit(X_train, y_train, epochs=50, batch_size=128, validation_split=0.1)
loss, accuracy = model.evaluate(X_test, y_test)
y_pred = model.predict(X_test)

print("Test Loss:", loss)
print("Test Accuracy:", accuracy)



y_pred_binary = np.round(y_pred).flatten()
print('----------------')
print('Confusion matrix')
print(confusion_matrix(y_test, y_pred_binary))
print('----------------')
print('Classification report')
print(classification_report(y_test, y_pred_binary))


2024-05-18 12:47:35.339516: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-05-18 12:47:35.532786: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


(35343, 100)


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 98, 64)         │           256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d (MaxPooling1D)    │ (None, 49, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 3136)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       401,536 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 401,921 (1.53 MB)

 Trainable params: 401,921 (1.53 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8647 - loss: 0.3180 - val_accuracy: 0.9233 - val_loss: 0.1973
Epoch 2/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9239 - loss: 0.1910 - val_accuracy: 0.9151 - val_loss: 0.1997
Epoch 3/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9345 - loss: 0.1725 - val_accuracy: 0.9219 - val_loss: 0.1917
Epoch 4/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9345 - loss: 0.1707 - val_accuracy: 0.9168 - val_loss: 0.2071
Epoch 5/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9337 - loss: 0.1685 - val_accuracy: 0.9307 - val_loss: 0.1811
Epoch 6/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9371 - loss: 0.1632 - val_accuracy: 0.9417 - val_loss: 0.1552
Epoch 7/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9428 - loss: 0.1498 - val_accuracy: 0.9477 - val_loss: 0.1497
Epoch 8/50
249/249 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9416 - loss: 0.1503 - val_accuracy: 0.